<a href="https://colab.research.google.com/github/shah23umar/kaggle/blob/master/customer_churn.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import matplotlib.style as style
import pandas as pd
style.use('seaborn-whitegrid')
%matplotlib inline

In [2]:
# Change format of charts to .svg
% config InlineBackend.figure_format = 'svg'

%xmode Plain

Exception reporting mode: Plain


In [3]:
df=pd.read_csv('/content/drive/My Drive/Retail/WA_Fn-UseC_-Telco-Customer-Churn.csv')

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


In [5]:
# Check for unique record ID #s - all were unique
df['customerID'].nunique()

7043

In [6]:
copies = df.duplicated(subset=['customerID','gender', 'SeniorCitizen', 'Partner', 'Dependents',
                               'tenure', 'PhoneService', 'MultipleLines', 'InternetService',
                               'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
                               'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling',
                               'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn'])

In [7]:
copies.value_counts()


False    7043
dtype: int64

In [8]:
# Drop this column since we no longer need it
df.drop('customerID', axis=1, inplace=True)

In [9]:
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [10]:
# Look at data to see if we need to recode any variables
df.head(2)

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,Female,0,Yes,No,1,No,No phone service,DSL,No,Yes,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,Male,0,No,No,34,Yes,No,DSL,Yes,No,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No


In [11]:
# Coded binary categories as 0 and 1 in preparation for regression
binary_dict = {'No': 0, 'No internet service': 0, 'No phone service': 0, 'Yes': 1}

def convert_cols_to_binary(col_names: list):
    for col in col_names:
        df[col] = df[col].map(binary_dict)


cols = ['Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
        'TechSupport', 'StreamingTV', 'StreamingMovies', 'PaperlessBilling', 'Churn']

convert_cols_to_binary(cols)

In [12]:
# Code gender as binary
gender_dict = {'Male': 0, 'Female': 1}
df['gender'] = df['gender'].map(gender_dict)

# Rename column to 'female' since 1's now represent females
df.rename(columns={'gender': 'female'})

,female,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,1,0,1,0,1,0,0,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0
1,0,0,0,0,34,1,0,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.5,0
2,0,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1
3,0,0,0,0,45,0,0,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0
4,1,0,0,0,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7038,0,0,1,1,24,1,1,DSL,1,0,1,1,1,1,One year,1,Mailed check,84.80,1990.5,0
7039,1,0,1,1,72,1,1,Fiber optic,0,1,1,0,1,1,One year,1,Credit card (automatic),103.20,7362.9,0
7040,1,0,1,1,11,0,0,DSL,1,0,0,0,0,0,Month-to-month,1,Electronic check,29.60,346.45,0
7041,0,1,1,0,4,1,1,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Mailed check,74.40,306.6,1


In [13]:
# Confirm that our column renamed correctly - it did
df.columns

Index(['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure',
       'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity',
       'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV',
       'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod',
       'MonthlyCharges', 'TotalCharges', 'Churn'],
      dtype='object')

In [14]:
# Recast binary columns as integers, as some of them were floats
df[['OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport',
    'StreamingTV', 'StreamingMovies']].astype('int')

,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,StreamingTV,StreamingMovies
0,0,1,0,0,0,0
1,1,0,1,0,0,0
2,1,1,0,0,0,0
3,1,0,1,1,0,0
4,0,0,0,0,0,0
...,...,...,...,...,...,...
7038,1,0,1,1,1,1
7039,0,1,1,0,1,1
7040,1,0,0,0,0,0
7041,0,0,0,0,0,0


Tried recasting TotalCharges to float to divide it by tenure, but got an error as there are 11 values in the column showing as ' '. This occurred because some people had not had any charges yet. I then dropped those records (only 11 of them) in order to allow us to cast the column as float.

In [15]:
# Dropping the 11 ' ' values from TotalCharges column
df = df[df['TotalCharges']!=' ']

# Then recast the column as float
df['TotalCharges'] = df['TotalCharges'].astype('float')

# Created new feature column to measure average monthly charges
df['AvgMonthlyCharges'] = df['TotalCharges']/df['tenure']

In [16]:
# Create dummy variables in preparation for regression.
# Drop first binary column for dummy variables to prevent redundant columns.
test = pd.get_dummies(data=df, columns=['InternetService', 'Contract', 'PaymentMethod'], drop_first=True)

In [17]:
df.columns = ['gender', 'senior', 'partner', 'dependents', 'tenure',
              'phone_service', 'multiple_lines', 'internet_service', 'online_security', 'online_backup',
              'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
              'contract', 'paperless_billing', 'payment_method', 'monthly_charges', 'total_charges', 'churn',
              'avg_monthly_charges']

test.columns = ['gender', 'senior', 'partner', 'dependents', 'tenure',
                'phone_service', 'multiple_lines', 'online_security', 'online_backup',
                'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
                'paperless_billing', 'monthly_charges', 'total_charges', 'churn',
                'avg_monthly_charges', 'internet_service-fiber_optic',
                'internet_service-no', 'contract-one_year', 'contract-two_year',
                'payment_method-credit_card_auto',
                'payment_method-electronic_check', 'payment_method-mailed_check']

Saving clean data and clean data + dummy variables for regression. Commenting code out below so we don't overwrite our current data files unless we mean to.



In [18]:
# # Save output as CSV
df.to_csv('/content/drive/My Drive/Retail/clean_data.csv', index=False)
test.to_csv('/content/drive/My Drive/Retail/clean_data_encoded_for_regression.csv', index=False)

**Feature engineering**

In [19]:
df = pd.read_csv('/content/drive/My Drive/Retail/clean_data.csv')


In [20]:
df.head()

,gender,senior,partner,dependents,tenure,phone_service,multiple_lines,internet_service,online_security,online_backup,device_protection,tech_support,streaming_tv,streaming_movies,contract,paperless_billing,payment_method,monthly_charges,total_charges,churn,avg_monthly_charges
0,1,0,1,0,1,0,0,DSL,0,1,0,0,0,0,Month-to-month,1,Electronic check,29.85,29.85,0,29.850000
1,0,0,0,0,34,1,0,DSL,1,0,1,0,0,0,One year,0,Mailed check,56.95,1889.50,0,55.573529
2,0,0,0,0,2,1,0,DSL,1,1,0,0,0,0,Month-to-month,1,Mailed check,53.85,108.15,1,54.075000
3,0,0,0,0,45,0,0,DSL,1,0,1,1,0,0,One year,0,Bank transfer (automatic),42.30,1840.75,0,40.905556
4,1,0,0,0,2,1,0,Fiber optic,0,0,0,0,0,0,Month-to-month,1,Electronic check,70.70,151.65,1,75.825000


In [21]:
df['monthly_charges'].describe()


count    7032.000000
mean       64.798208
std        30.085974
min        18.250000
25%        35.587500
50%        70.350000
75%        89.862500
max       118.750000
Name: monthly_charges, dtype: float64

In [22]:
df['tenure'].describe()


count    7032.000000
mean       32.421786
std        24.545260
min         1.000000
25%         9.000000
50%        29.000000
75%        55.000000
max        72.000000
Name: tenure, dtype: float64

In [23]:
df['internet_service'].value_counts()


Fiber optic    3096
DSL            2416
No             1520
Name: internet_service, dtype: int64

In [24]:
df.groupby(df['internet_service'])['monthly_charges'].mean()


internet_service
DSL            58.088017
Fiber optic    91.500129
No             21.076283
Name: monthly_charges, dtype: float64

In [25]:
x = df.groupby(df['tenure'])['monthly_charges'].mean()


In [26]:
df.groupby(df['phone_service'])['monthly_charges'].mean()


phone_service
0    41.992500
1    67.239625
Name: monthly_charges, dtype: float64

**Use Linear Regression to predict how much customers should be paying**

In [27]:
import pprint
import matplotlib.pyplot as plt
import matplotlib.style as style
import pandas as pd
import seaborn as sns
import sklearn.metrics
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split

style.use('seaborn-whitegrid')
% matplotlib inline

/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [28]:
df = pd.read_csv('/content/drive/My Drive/Retail/clean_data_encoded_for_regression.csv')


In [29]:
sns.heatmap(df, cmap='coolwarm', )


In [30]:
X2 = df[['phone_service',
       'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies', 'internet_service-fiber_optic',
       'internet_service-no']]

y2 = df['monthly_charges']

In [31]:
X2_train, X2_test, y2_train, y2_test = train_test_split(X2, y2, test_size=0.33, random_state=22)


In [ ]:
lr = LinearRegression()
lr.fit(X2_train, y2_train)
predictions = lr.predict(X2_test)

In [ ]:
lr.score(X2_test, y2_test)


It looks like the cost of services is nearly 100% linear

In [ ]:
pprint.pprint(list(zip(lr.coef_, X2.columns)))


In [ ]:
df['predicted_monthly_charges'] = lr.predict(X2)


In [ ]:
df.head()


In [ ]:
df['predicted_minus_actual_monthly_charges'] = df['monthly_charges'] - df['predicted_monthly_charges']


In [ ]:
df.head()


In [ ]:
# Total revenue per month, all customers
df['total_charges'].sum()

In [ ]:
df[df['churn'] == 1]['total_charges'].sum()


In [ ]:
# Total revenue per month, all customers
df['monthly_charges'].sum()

In [ ]:
df[df['churn'] == 1]['monthly_charges'].sum()


In [ ]:
139130.85/455661.0

In [ ]:
df['avg_monthly_charges'].mean()


**Regression**

In [46]:
import pickle
import pprint
import matplotlib.pyplot as plt
import matplotlib.style as style
import numpy as np
import pandas as pd
from imblearn.over_sampling import SMOTE
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier, AdaBoostClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import roc_auc_score, roc_curve, classification_report, confusion_matrix, recall_score
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_predict, KFold
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import Pipeline
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier

In [47]:
df = pd.read_csv('/content/drive/My Drive/Retail/clean_data_encoded_for_regression.csv')

X = df[['gender', 'senior', 'partner', 'dependents', 'tenure', 'phone_service',
       'multiple_lines', 'online_security', 'online_backup',
       'device_protection', 'tech_support', 'streaming_tv', 'streaming_movies',
       'paperless_billing', 'monthly_charges', 'total_charges',
       'avg_monthly_charges', 'internet_service-fiber_optic',
       'internet_service-no', 'contract-one_year', 'contract-two_year',
       'payment_method-credit_card_auto', 'payment_method-electronic_check',
       'payment_method-mailed_check']]

y = df['churn']

# Stratify our train-test-split so that we have a balanced split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=40, stratify=y)

**Upsample with SMOTE to get a balanced dataset**

Currently, only 27% of our dataset is categorized as "Churn" (i.e., will leave the telecom company and no longer be a customer this month). In order to train our model better, we are upsampling with SMOTE so that 50% of our dataset is categorized as "Churn". This makes sense because we are more concerned about recall (making sure our model catches nearly everyone who is likely to churn) than it is about precision.

The rationale? It's much cheaper to keep an existing customer than to acquire a new one.

In [48]:
sm = SMOTE(random_state=42, ratio=1.0)
X_train_res, y_train_res = sm.fit_sample(X_train, y_train)
print(y_train.value_counts(), np.bincount(y_train_res))

0    3614
1    1308
Name: churn, dtype: int64 [3614 3614]


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function safe_indexing is deprecated; safe_indexing is deprecated in version 0.22 and will be removed in version 0.24.
  warnings.warn(msg, category=FutureWarning)


Let's do some quick and dirty analyses to determine which models we should tune further


First we'll do ANOTHER train test split on the training set. We want to make sure that we leave our actual test data (X_test and y_test) alone.



In [49]:
X_train_2, X_test_2, y_train_2, y_test_2 = train_test_split(X_train_res, y_train_res, test_size=0.33, random_state=20)


In [50]:
def quick_test(model, X, y):
    xtrain, xtest, ytrain, ytest = train_test_split(X, y, test_size=0.3)
    model.fit(xtrain, ytrain)
    return model.score(xtrain, ytrain), model.score(xtest, ytest)


def quick_test_a_few_times(model, X, y, n=10):
    return (pd.DataFrame([quick_test(model, X, y) for j in range(n)],
                         columns=['Train Acc', 'Test Acc'])).mean()

new_model = make_pipeline(StandardScaler(), LinearSVC(max_iter=10000))
# pred = new_model.predict(X_test)
# Do the test 10 times with a LinearSVC and get the average score
print(quick_test_a_few_times(new_model, X_train_2, y_train_2))

Train Acc    0.770493
Test Acc     0.766345
dtype: float64


In [51]:
def get_scores(model):
    mod = model
    mod = mod.fit(X_train_2, y_train_2)
    predictions = mod.predict(X_test_2)
    # quick_test_a_few_times(decisiontree, X_train, y_train)

    print(f'**Model: {mod}**')
    print(f'Train accuracy: {mod.score(X_train_2, y_train_2)}')
    print(f'Test accuracy: {mod.score(X_test_2, y_test_2)}')
    print(f'Precision: {precision_score(y_test_2, predictions)}')
    print(f'Recall: {recall_score(y_test_2, predictions)}')
    print()
    print(classification_report(y_test_2, predictions))
    print()

**Decision Tree**

In [53]:
from sklearn.metrics import precision_score


In [54]:
def evaluate_models(model_types: list):
    for model in model_types:
        m = model_types[model]
        quick_test_a_few_times(X_train_2, y_train_2)
        get_scores(m)


models = [
    {
        'label': 'Logistic Regression',
        'model': LogisticRegression(),
    },
    {
        'label': 'Gradient Boosting',
        'model': GradientBoostingClassifier(),
    },
    {
        'label': 'Random Forest',
        'model': RandomForestClassifier(),
    },
    {
        'label': 'AdaBoost Classifier',
        'model': AdaBoostClassifier(),
    },
    {
        'label': 'Support Vector Classifier',
        'model': SVC(probability=True),
    }
]

# evaluate_models(models)

decision_tree = DecisionTreeClassifier()
quick_test_a_few_times(decision_tree, X_train_2, y_train_2)
get_scores(decision_tree)

**Model: DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None, criterion='gini',
                       max_depth=None, max_features=None, max_leaf_nodes=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, presort='deprecated',
                       random_state=None, splitter='best')**
Train accuracy: 0.9993804213135068
Test accuracy: 0.7967309304274938
Precision: 0.7925
Recall: 0.8011794439764112

              precision    recall  f1-score   support

           0       0.80      0.79      0.80      1199
           1       0.79      0.80      0.80      1187

    accuracy                           0.80      2386
   macro avg       0.80      0.80      0.80      2386
weighted avg       0.80      0.80      0.80      2386




Random Forests

In [55]:
random_forest = RandomForestClassifier()
quick_test_a_few_times(random_forest, X_train_2, y_train_2)
get_scores(random_forest)

**Model: RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=None,
                       verbose=0, warm_start=False)**
Train accuracy: 0.9993804213135068
Test accuracy: 0.855406538139145
Precision: 0.868006993006993
Recall: 0.8365627632687447

              precision    recall  f1-score   support

           0       0.84      0.87      0.86      1199
           1       0.87      0.84      0.85      1187

    accuracy                           0.86      2386
   macro avg       0.86      0.86      0.86      2386
weighted avg       0.86      0.86      0.86  

Gradient Boosting Classifier

In [56]:
grad_boost = GradientBoostingClassifier()
quick_test_a_few_times(grad_boost, X_train_2, y_train_2)
get_scores(grad_boost)

**Model: GradientBoostingClassifier(ccp_alpha=0.0, criterion='friedman_mse', init=None,
                           learning_rate=0.1, loss='deviance', max_depth=3,
                           max_features=None, max_leaf_nodes=None,
                           min_impurity_decrease=0.0, min_impurity_split=None,
                           min_samples_leaf=1, min_samples_split=2,
                           min_weight_fraction_leaf=0.0, n_estimators=100,
                           n_iter_no_change=None, presort='deprecated',
                           random_state=None, subsample=1.0, tol=0.0001,
                           validation_fraction=0.1, verbose=0,
                           warm_start=False)**
Train accuracy: 0.8764973151590252
Test accuracy: 0.8562447611064543
Precision: 0.8637931034482759
Recall: 0.8441449031171019

              precision    recall  f1-score   support

           0       0.85      0.87      0.86      1199
           1       0.86      0.84      0.85      1187

 

AdaBoost Classifier

In [58]:
ada_boost = AdaBoostClassifier()
quick_test_a_few_times(ada_boost, X_train_2, y_train_2)
get_scores(ada_boost)

**Model: AdaBoostClassifier(algorithm='SAMME.R', base_estimator=None, learning_rate=1.0,
                   n_estimators=50, random_state=None)**
Train accuracy: 0.8597686906237092
Test accuracy: 0.8365465213746857
Precision: 0.8379983036471587
Recall: 0.8323504633529908

              precision    recall  f1-score   support

           0       0.84      0.84      0.84      1199
           1       0.84      0.83      0.84      1187

    accuracy                           0.84      2386
   macro avg       0.84      0.84      0.84      2386
weighted avg       0.84      0.84      0.84      2386




Gradient Boosting Classifier is winning model